In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, LongType, MapType, StringType

spark = (
    SparkSession.builder
    .appName("CryptoConsumerNotebook")
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.2,org.postgresql:postgresql:42.7.3")
    .getOrCreate()
)

schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("prices", MapType(StringType(), MapType(StringType(), StringType())), True),
])

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0cd570ca-45a2-4357-9e35-e6c03615f4db;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.2 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.postgresql#postgr

In [14]:
from pyspark.sql.functions import from_unixtime

# Schema da mensagem que você já está produzindo
schema = StructType([
    StructField("timestamp", LongType(), True),
    StructField("prices", MapType(StringType(), MapType(StringType(), StringType())), True)
])

# Lê do Kafka
df = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka:29092")
    .option("subscribe", "crypto_prices")
    .option("startingOffsets", "latest")
    .load()
)

# Converte JSON
json_df = df.select(
    from_json(col("value").cast("string"), schema).alias("data")
)

flattened = json_df.select(
    from_unixtime(col("data.timestamp")).cast("timestamp").alias("ts"),
    col("data.prices.bitcoin.usd").cast("double").alias("btc_usd"),
    col("data.prices.ethereum.usd").cast("double").alias("eth_usd")
)

In [15]:
import os

user  = os.getenv("POSTGRES_USER")
pwd = os.getenv("POSTGRES_PASSWORD")
db = os.getenv("POSTGRES_DB")
# Grava no Postgres (tabela crypto_prices)
query = (
    flattened.writeStream
    .foreachBatch(lambda batch_df, batch_id: (
        batch_df.write
        .format("jdbc")
        .option("url", f"jdbc:postgresql://postgres:5432/{db}")
        .option("dbtable", "crypto_prices")
        .option("user", user)
        .option("password", pwd)
        .option("driver", "org.postgresql.Driver")
        .mode("append")
        .save()
    ))
    .outputMode("update")
    .start()
)

query.awaitTermination()

25/08/29 00:44:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9c584896-172f-4ed9-9f0e-bc7c428aadb5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/29 00:44:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/08/29 00:44:31 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/opt/bitnami/spark/python/lib/py4j.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
             

KeyboardInterrupt: 